# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 12:24PM,247157,10,0086003230,ISDIN Corporation,Released
1,Sep 23 2022 12:24PM,247157,10,0086003277,ISDIN Corporation,Released
2,Sep 23 2022 12:24PM,247157,10,0086003287,ISDIN Corporation,Released
3,Sep 23 2022 12:24PM,247157,10,0086003308,ISDIN Corporation,Released
4,Sep 23 2022 12:24PM,247157,10,0086003335,ISDIN Corporation,Released
5,Sep 23 2022 12:24PM,247157,10,0086005845,ISDIN Corporation,Released
6,Sep 23 2022 12:24PM,247157,10,0086005852,ISDIN Corporation,Released
7,Sep 23 2022 12:24PM,247157,10,0086005858,ISDIN Corporation,Released
8,Sep 23 2022 12:24PM,247157,10,0086005866,ISDIN Corporation,Released
9,Sep 23 2022 12:22PM,247159,19,ACG-2102491330,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247152,Released,2
26,247156,Released,4
27,247157,Released,9
28,247158,Released,3
29,247159,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247152,NaN,NaN,2.0
247156,NaN,NaN,4.0
247157,NaN,NaN,9.0
247158,NaN,NaN,3.0
247159,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
246999,0.0,1.0,0.0
247000,0.0,1.0,0.0
247037,0.0,0.0,34.0
247083,5.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
246999,0,1,0
247000,0,1,0
247037,0,0,34
247083,5,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,246999,0,1,0
2,247000,0,1,0
3,247037,0,0,34
4,247083,5,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,246999,,1,
2,247000,,1,
3,247037,,,34
4,247083,5,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation
9,Sep 23 2022 12:22PM,247159,19,ACG North America LLC
10,Sep 23 2022 12:21PM,247158,10,Hush Hush
13,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc."
47,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC"
51,Sep 23 2022 12:00PM,247152,10,Emerginnova
53,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation
76,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation
89,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc."
106,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,,,9
1,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,,,1
2,Sep 23 2022 12:21PM,247158,10,Hush Hush,,,3
3,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",,,34
4,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",,,4
5,Sep 23 2022 12:00PM,247152,10,Emerginnova,,,2
6,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,,,23
7,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,,,13
8,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",,,17
9,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9,,
1,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1,,
2,Sep 23 2022 12:21PM,247158,10,Hush Hush,3,,
3,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",34,,
4,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",4,,
5,Sep 23 2022 12:00PM,247152,10,Emerginnova,2,,
6,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,23,,
7,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,13,,
8,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",17,,
9,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9,,
1,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1,,
2,Sep 23 2022 12:21PM,247158,10,Hush Hush,3,,
3,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",34,,
4,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9.0,NaN,NaN
1,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1.0,NaN,NaN
2,Sep 23 2022 12:21PM,247158,10,Hush Hush,3.0,NaN,NaN
3,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",34.0,NaN,NaN
4,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9.0,0.0,0.0
1,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1.0,0.0,0.0
2,Sep 23 2022 12:21PM,247158,10,Hush Hush,3.0,0.0,0.0
3,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",34.0,0.0,0.0
4,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2965678,77.0,5.0,7.0
15,741310,55.0,0.0,0.0
16,494253,2.0,0.0,0.0
19,988535,5.0,1.0,0.0
20,247143,6.0,0.0,0.0
21,1235121,2.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2965678,77.0,5.0,7.0
1,15,741310,55.0,0.0,0.0
2,16,494253,2.0,0.0,0.0
3,19,988535,5.0,1.0,0.0
4,20,247143,6.0,0.0,0.0
5,21,1235121,2.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,5.0,7.0
1,15,55.0,0.0,0.0
2,16,2.0,0.0,0.0
3,19,5.0,1.0,0.0
4,20,6.0,0.0,0.0
5,21,2.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,15,Released,55.0
2,16,Released,2.0
3,19,Released,5.0
4,20,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,0.0,1.0,0.0,3.0
Released,77.0,55.0,2.0,5.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,3.0
2,Released,77.0,55.0,2.0,5.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,3.0
2,Released,77.0,55.0,2.0,5.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()